### Fetching Ugandan news data

**Import the `BeautifulSoup` library for web scraping and `requests` for making web requests**

In [72]:
import re
import requests
from bs4 import BeautifulSoup
import datetime

**Setting up `pymongo`, a package used to connect with `MongoDB`, where the news data will be stored**

In [73]:
import pymongo
client = pymongo.MongoClient()
news_db = client["ug_news"]
news_col = news_db["news"]
news_db.news.create_index([('slug', pymongo.ASCENDING)], unique=True)

'slug_1'

In [74]:
def clean_article_text(paragraphs):
    del paragraphs[0:2]
    del paragraphs[-7:]
    cleaned_text = []
    for p in paragraphs:
        p_text = p.get_text().strip()
        cleaned_text.append(p_text)
    return " ".join(cleaned_text)

**Define and call a method for fetching data from the `New Vision` online newspaper**

In [75]:
def fetch_new_vision():
    """Fetch data from the New Vision online newspaper"""
    
    # Fetch data from the New Vision online coverpage
    url = "https://www.newvision.co.ug"
    news_href = re.compile(r"https://www.newvision.co.ug/new_vision/news/")

    news_request = requests.get(url)
    coverpage = news_request.content
    
    # Pick out the anchor tags to get the links to the actual news articles
    soup1 = BeautifulSoup(coverpage, 'html5lib')
    article_links = soup1.find_all(href=news_href)
    
    
    # Follow each link and fetch the article content
    all_articles = []

    for link in article_links:
        article = requests.get(link['href'])
        article_content = article.content
        soup2 = BeautifulSoup(article_content, 'html5lib')
        title = soup2.find('h1').get_text()
        slug = "-".join(title.split())
        paragraphs = soup2.find_all('p', recursive=True)
        cleaned_article = clean_article_text(paragraphs)
        all_articles.append({'slug': slug, 'text': cleaned_article})
            
    return all_articles

nv_news = fetch_new_vision()
nv_news

[{'slug': '10-perish-in-fresh-Kasese-floods',
  'text': 'The victims drowned when the river, which marks the common border along Kitholhu sub-county, burst its banks following overnight torrential rains in Mt Rwenzori. DISASTER Five people from one family are confirmed dead in a new wave of floods\xa0that hit Bukonzo West Constituency, long the Uganda-DR Congo border\xa0today morning. Meanwhile, on the Congo side, five other civilians also died\xa0when river Lhubiriha overran their community along the common border. The victims drowned when the river, which marks the common border\xa0along Kitholhu sub-county, burst its banks following overnight\xa0torrential rains in Mt Rwenzori. One UPDF is reportedly missing while\xa0three others are admitted at Bwera referral hospital after they narrowly\xa0survived when the river invaded the area.\xa0Find the details of this as well as other flooding incidents in the New Vision tomorrow. Click on Epaper for your copy Rwenzururu Queen ends visit to

**Define and call a method for fetching data from the `Daily Monitor` online newspaper**

In [67]:
def fetch_daily_monitor():
    pass

fetch_daily_monitor()

**Store article text in `mongodb` database**

In [69]:
news_col.insert_many(nv_news)

BulkWriteError: batch op errors occurred

In [37]:
# col.insert_many(daily_monitor_news)